Cheng Shi;
USCID: 8556138235;
Github:CaronShi

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, accuracy_score
from imblearn.over_sampling import RandomOverSampler
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from imblearn.pipeline import Pipeline as imbpipeline
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

1. Tree-Based Method

In [57]:
#(a)

train = pd.read_csv("../data/aps_failure_training_set.csv", skiprows=20).replace("na", np.nan)
test = pd.read_csv("../data/aps_failure_test_set.csv", skiprows=20).replace("na", np.nan)

#seperate dataset 
train_x = train.iloc[:, 1:]#datapoint without class
train_y = train.iloc[:, 0]#class
test_x = test.iloc[:,1:]
test_y = test.iloc[:, 0]

(b)Data Preparation

i.Research what types of techniques are usually used for dealing with data with missing values. Pick at least one of them and apply it to this data in the next steps.

According to what we learned in class, there are some of the methods dealing with missing value. 1)replace missing values by median/mean or other meaningful statistics. 2)using some models to predict the misssing value 3)simply skip the missing values 4)replacing by most frequent values. In this data set, since we have a lot of missing values, it's not optimal to skip those missing data, I decide to choose using median replacing methods and most frequent number replacing method in this question.

In [ ]:
from sklearn.impute import SimpleImputer


imp = SimpleImputer(missing_values = np.nan, strategy = "median")
train_x = pd.DataFrame(imp.fit_transform(train_x), columns = train_x.columns)
test_x = pd.DataFrame(imp.fit_transform(test_x), columns = test_x.columns)
train_x

ii.For each of the 170 features, calculate the coefficient of variation CV=s/m,where s is sample standard deviation and m is sample mean.

In [ ]:
cv = train_x.std() / train_x.mean()
cv

iii.Plot a correlation matrix for your features using pandas or any other tool.

In [ ]:
cols_df = train_x.columns
cols = []
for i in cols_df:
    cols.append(i)

fig = plt.figure(figsize=(20,20))
plt.matshow(train_x.corr(),fignum= fig.number)
plt.xticks(range(len(cols)),cols,rotation='vertical')
plt.yticks(range(len(cols)),cols)
plt.title('Correlation Matrix')
plt.show()


iv.Pick √170 features with highest CV, and make scatter plots and box plots for them, similar to those on p.129 of ISLR. Can you draw conclusions about significance of those features, just by the scatter plots? This does not mean that you will only use those features in the following questions. We picked them only for visualization.

Yes, from the graphs, we can see a positive relationship in au000 and as000 so we may say au000 and as000 are significant features.

In [ ]:
pick = np.sqrt(170)
pick = np.floor(pick)
cv_pick = cv.sort_values(ascending=False) [:int(pick)]
cv_pick

In [ ]:
pick = np.sqrt(170)
pick = np.floor(pick)

cv_pick = cv.sort_values(ascending=False) [:int(pick)]
df_pick = pd.DataFrame(train_x, columns = cv_pick.keys())
df_pick.loc[:, "class"] = train_y
sns.pairplot(data = df_pick, kind = "scatter", diag_kind= 'none')

In [ ]:
# box plots
for y in list(cv_pick.keys()):
    fig = plt.figure(figsize = (3, 3))
    sns.boxplot(data = df_pick, x="class", y=y, showfliers = False)
    plt.tight_layout()
    plt.show()
    

In [ ]:
boxplot = df_pick.boxplot(rot=90, fontsize=10 )

v.Determine the number of positive and negative data. Is this data set imbalanced?

ANS:
From the next cell, we can see that the  number of negatives is far more than positives' in both train(59000 vs 1000) and test(15625 vs 375) dataset, so we could conclude the dataset is imbalanced.

In [ ]:
print("Train:",train_y.value_counts())

print("Test:",test_y.value_counts())


(c)Train a random forest to classify the data set. Do NOT compensate for class imbalance in the data set. 

Calculate the confusion matrix, ROC, AUC, and misclassification for training and test sets and report them (You may use pROC package). Calculate Out of Bag error estimate for your random forset and compareit to the test error.

In [ ]:
#train a random forest
y_true = train_y.replace("neg", 0).replace("pos", 1)
#test_y = test_y.replace("neg", 0).replace("pos", 1)
clf = RandomForestClassifier(oob_score = True)
clf.fit(train_x, train_y)
y_pred = clf.predict(train_x)


In [ ]:
#y_pred = y_pred.replace("neg", 0).replace("pos", 1)
#confusion matrix
cm = confusion_matrix(train_y, y_pred)
print('confusion_matrix:', cm)

#misclassification
tn, fp, fn, tp = cm.ravel()
misclassfication = fp+fn
print('misclassification:',misclassfication)

print('misclassification rate :',misclassfication/(tn+fp+fn+tp) )

#Out of Bag error estimate
oob_error = 1 - clf.oob_score_
print('oob error:',oob_error)

#ROC
scores = clf.predict_proba(train_x)[:, 1]
fpr, tpr, thresholds = roc_curve(y_true, scores)

#AUC
auc = roc_auc_score(y_true, scores)
print('aur:', auc)
plt.plot(fpr, tpr,label='ROC')
plt.plot([0, 1], [0, 1], color="black",label = 'AUC')
plt.title("ROC curve")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.legend()
plt.show()


In [ ]:
#test a random forest
test_y = test_y.replace(0,"neg").replace(1,"pos")
y_pred = clf.predict(test_x)
#confusion matrix
cm = confusion_matrix(test_y, y_pred)
print('confusion_matrix:', cm)

#misclassification
tn, fp, fn, tp = cm.ravel()
misclassfication = fp+fn
print('misclassification:',misclassfication )
print('misclassification rate :',misclassfication/(tn+fp+fn+tp) )
#Out of Bag error estimate
oob_error = 1 - clf.oob_score_
print('oob error:',oob_error)

#ROC
y_true= test_y.replace("neg", 0).replace("pos", 1)
scores = clf.predict_proba(test_x)[:, 1]
fpr, tpr, thresholds = roc_curve(y_true, scores)

#AUC
auc = roc_auc_score(y_true, scores)
print('aur:', auc)
plt.plot(fpr, tpr,label='ROC')
plt.plot([0, 1], [0, 1], color="black",label = 'AUC')
plt.title("ROC curve")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.legend()
plt.show()



(d) Research how class imbalance is addressed in random forests. Compensate for class imbalance in your random forest and repeat 1c. Compare the results withthose of 1c.4

ANS:
Random forests address imbalance with class weighting and random undersampling.

reference: https://machinelearningmastery.com/bagging-and-random-forest-for-imbalanced-classification/



In [ ]:
clf = RandomForestClassifier(oob_score = True,class_weight='balanced')

clf.fit(train_x, train_y)
y_pred = clf.predict(train_x)

In [ ]:
#train
y_true = train_y.replace("neg", 0).replace("pos", 1)
#confusion matrix
cm = confusion_matrix(train_y, y_pred)
print('confusion_matrix:', cm)

#misclassification
tn, fp, fn, tp = cm.ravel()
misclassfication = fp+fn
print('misclassification:',misclassfication)

print('misclassification rate :',misclassfication/(tn+fp+fn+tp) )

#Out of Bag error estimate
oob_error = 1 - clf.oob_score_
print('oob error:',oob_error)

#ROC
scores = clf.predict_proba(train_x)[:, 1]
fpr, tpr, thresholds = roc_curve(y_true, scores)

#AUC
auc = roc_auc_score(y_true, scores)
print('aur:', auc)
plt.plot(fpr, tpr,label='ROC')
plt.plot([0, 1], [0, 1], color="black",label = 'AUC')
plt.title("ROC curve")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.legend()
plt.show()

In [ ]:
#test a random forest
y_true= test_y.replace("neg", 0).replace("pos", 1)
y_pred = clf.predict(test_x)
#confusion matrix
cm = confusion_matrix(test_y, y_pred)
print('confusion_matrix:', cm)

#misclassification
tn, fp, fn, tp = cm.ravel()
misclassfication = fp+fn
print('misclassification:',misclassfication )
print('misclassification rate :',misclassfication/(tn+fp+fn+tp) )
#Out of Bag error estimate
oob_error = 1 - clf.oob_score_
print('oob error:',oob_error)

#ROC
scores = clf.predict_proba(test_x)[:, 1]
fpr, tpr, thresholds = roc_curve(y_true, scores)

#AUC
auc = roc_auc_score(y_true, scores)
print('aur:', auc)
plt.plot(fpr, tpr,label='ROC')
plt.plot([0, 1], [0, 1], color="black",label = 'AUC')
plt.title("ROC curve")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.legend()
plt.show()



In [ ]:
d = {'oob error':[0.00633,0.00765,0.00633,0.00765],'misclassification':[0,0,126,182]
    }

compare = pd.DataFrame(data = d)
compare.index = ['train unbalance','train balance','test unbalance','test balance']
print(compare)

From the table above, we can observe the difference between class balanced and unbalanced, and for the obb error, the balanced one has a larger error; for the misclassification number, the balanced one for test set also get larger.

(e)XGBoost and Model Trees

use XGBoost to fit the model tree. 
 
determine α(the regularization term) using cross-validation. 
 
Train the model for the APS data set without compensation for class imbalance. 
 
Use one of 5 fold, 10 fold, and leave-one-out cross validation methods to estimate the error of your trained model and compare it with the test error. 
 
Report the Confusion Matrix, ROC,and AUC for training and test sets.

In [ ]:
y_true = train_y.replace("neg", 0).replace("pos", 1)

#use XGBoost to fit the model tree
xgb = XGBClassifier(objective= "binary:logistic", use_label_encoder=False, )

#5-fold
param_grid = {'reg_alpha':[1e-3, 1e-2, 0.1, 1, 10, 100]}
gsearch = GridSearchCV(estimator = xgb, param_grid = param_grid, scoring = "roc_auc", cv = 5)
gsearch.fit(train_x, y_true, eval_metric = "logloss")
print('the best score for 5-fold cv:',gsearch.best_score_)
print('the best alpha for 5-fold cv: ',gsearch.best_params_)

the best score for 5-fold cv: 0.9907702542372882
the best alpha for 5-fold cv:  {'reg_alpha': 10}

In [ ]:
y_true= train_y.replace("neg", 0).replace("pos", 1)
test_y = test_y.replace("neg", 0).replace("pos", 1)
xgb_model = XGBClassifier(objective= "binary:logistic", random_state = 0,  eval_metric = "logloss",use_label_encoder = False, reg_alpha = 10)
#xgbc.fit(train_x, y_true)
xgb_model.fit(train_x, y_true)
predict_y = xgb_model.predict(test_x)
test_error = 1 - accuracy_score(test_y, predict_y)
print("test error:", test_error)


In [ ]:
xgb_score = 1-test_error
d = {'test error':[xgb_score,0.990770],'misclas':[0,0]}

compare = pd.DataFrame(data = d)
compare.index = ['xgbc','5-fold cv']
print(compare)

From the table above, the test score is bigger than the 5-fold score

In [ ]:
#train  dataset
y_true = train_y.replace("neg", 0).replace("pos", 1)
test_y = test_y.replace("neg", 0).replace("pos", 1)
y_pred = xgb_model.predict(train_x)

#confusion matrix
cm = confusion_matrix(y_true, y_pred)
print('confusion_matrix:', cm)


#ROC
scores = clf.predict_proba(train_x)[:, 1]
fpr, tpr, thresholds = roc_curve(y_true, scores)

#AUC
auc = roc_auc_score(y_true, scores)
print('aur:', auc)
plt.plot(fpr, tpr,label='ROC')
plt.plot([0, 1], [0, 1], color="black",label = 'AUC')
plt.title("ROC curve")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.legend()
plt.show()


In [ ]:
#test dataset
y_true= test_y.replace("neg", 0).replace("pos", 1)
y_pred = xgb_model.predict(test_x)

#confusion matrix
cm = confusion_matrix(y_true, y_pred)
print('confusion_matrix:', cm)

#ROC
scores = clf.predict_proba(test_x)[:, 1]
fpr, tpr, thresholds = roc_curve(y_true, scores)

#AUC
auc = roc_auc_score(y_true, scores)
print('aur:', auc)
plt.plot(fpr, tpr,label='ROC')
plt.plot([0, 1], [0, 1], color="black",label = 'AUC')
plt.title("ROC curve")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.legend()
plt.show()


(f)Use SMOTE (Synthetic Minority Over-sampling Technique) to pre-process your data to compensate for class imbalance. Train XGBoost with L1-penalized logistic regression at each node using the pre-processed data and repeat 1e. Do not forget that there is a right and a wrong way of cross validation here. Comparethe uncompensated case with SMOTE case.

reference https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/

In [ ]:
pipeline = imbpipeline(steps = [['smote', SMOTE(random_state = 0)], ['scaler', MinMaxScaler()],
                                ['classifier', LogisticRegression(random_state = 0, max_iter=1000)]])
param_grid = {"classifier__C":[1e-3, 1e-2, 0.1, 1, 10, 100]}
stratified_kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)

train_y_num = train_y.replace("neg", 0).replace("pos", 1)

#trainy = train_y.replace("neg", 0).replace("pos", 1)
#trainx = train_x.replace("neg", 0).replace("pos", 1)
# 5-fold cv with SMOTE
grid_search = GridSearchCV(estimator = pipeline, param_grid = param_grid, scoring = "roc_auc", cv = stratified_kfold, n_jobs=-1)
grid_search.fit(train_x, train_y_num)

In [ ]:
gparam = grid_search.best_params_, 
gs_score = grid_search.best_score_
gtest_score = grid_search.score(test_x,test_y)
print(gparam,'best score:', gs_score,'test score:', gtest_score)

the test score 0.9891758, which is smaller 5-fold cv test error 0.99077 and xgbc test error 0.993437

In [ ]:
y_pred = grid_search.predict(train_x)
y_true= train_y.replace("neg", 0).replace("pos", 1)
#confusion matrix
cm = confusion_matrix(y_true, y_pred)
print('confusion_matrix:', cm)


#ROC
scores_tr = grid_search.predict_proba(train_x)[:, 1]
fpr, tpr, thresholds = roc_curve(y_true, scores_tr)

#AUC
auc = roc_auc_score(y_true, scores_tr)
print(auc)
plt.plot(fpr, tpr,label='ROC')
plt.plot([0, 1], [0, 1], color="black",label = 'AUC')
plt.title("ROC curve")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.legend()
plt.show()

In [ ]:
#test
y_pred = grid_search.predict(test_x)
y_true= test_y.replace("neg", 0).replace("pos", 1)
#confusion matrix
cm = confusion_matrix(y_true, y_pred)
print('confusion_matrix:', cm)


#ROC
scores_test = grid_search.predict_proba(test_x)[:, 1]
fpr, tpr, thresholds = roc_curve(y_true, scores_test)

#AUC
auc = roc_auc_score(y_true, scores_test)
print(auc)
plt.plot(fpr, tpr,label='ROC')
plt.plot([0, 1], [0, 1], color="black",label = 'AUC')
plt.title("ROC curve")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.legend()
plt.show()

ISLR 6.6.3

(a) iv, Steadily decrease; because we need increase s and make models more flexiable.

(b) ii,Decrease initially, and then eventually start increasing in a U shape; because when models more flexiable, RSS will reduce then increase while overfitting
(c) iii, Steadily increase; variance will increase with flexiblility.
(d) iv, Steadily decrease; bias decreased with the increasing flexiblility.
(e) v Remain constant, because it is independent and not be affected by other factors

ISLR 6.6.5

(a)ridge = (y1−β1x11−β2x12)^2+(y2−β1x21−β2x22)^2+λ(β1^2+β2^2)

(b)because we know  x11=x12,x21=x22,y1+y2=0 and x11+x21=0 and x12+x22=0,
x11=x12 =-x21=-x22=x, y1=-y2=y,
simplied formulas: 2[y-x(β1+β2)]^2 + λ(β1^2+β2^2) = 2y^2-4yxβ1-4yxβ2+ [x(β1+β2)]^2 +λ(β1^2+β2^2) = 2y^2-4yxβ1-4yxβ2+ x^2 *(β1^2+2β1β2+β2^2)+λ(β1^2+β2^2)  = 0 

λ2β1=4x[y−x(β1+β2)]
λ2β2=4x[x−y(β1+β2)]
so β1=β2

(c)lasso = (y1−β1x11−β2x12)^2+(y2−β1x21−β2x22)^2+λ(|β1|+|β2|)

(d)the only difference between lasso formula and ridge's is the loss penalty function after λ, so we could easily find the simplied formula for lasso is 2[y-x(β1+β2)]^2 + λ(|β1|+|β2|) , opt. = β1+β2= y/x= s, if in f(x,y) = 4x[x−y(β1+β2)], β1+β2 is +, f(x,y) is -, and β1+β2 is -, f(x,y) is +, so the lasso doesn't have only one solution.

ISLR 8.4.5

(1) Majority vote: it's red because the number is 6 out of 10;
(2)Avg: the mean is 0.45 which belongs to green.


ISLR 9.7.3

(a) We are given n = 7 observations in p = 2 dimensions. For each observation, there is an associated class label. Sketch the observations.

In [ ]:
x1 = [3, 2, 4, 1, 2, 4, 4]
x2 = [4, 2, 4, 4, 1, 3, 1]
y = ["red", "red", "red", "red", "blue", "blue", "blue"]
plt.scatter(x1, x2, c = y)
plt.show()

(b) Sketch the optimal separating hyperplane, and provide the equation for this hyperplane (of the form (9.1)). 

ANS:X1−X2+0.5 = 0

In [ ]:
plt.scatter(x1, x2, c = y)
x = np.arange(0, 5, 0.1)
y = x - 0.5
plt.plot(x,y)
plt.show()

(c) Describe the classification rule for the maximal margin classifier. It should be something along the lines of “Classify to Red if β0+β1X1+β2X2>0, and classify to Blue otherwise.” Provide the values for β0,β1,β2.

if X2 - X1 + 0.5 > 0, it's in red class, or in blue class when < 0 

(d)On your sketch, indicate the margin for the maximal margin hyperplane.

In [ ]:
y = ["red", "red", "red", "red", "blue", "blue", "blue"]
plt.plot([1,5],[0,4], linestyle = '--')
plt.plot([0.5, 5], [0, 4.5])
plt.plot([0,5],[0,5], linestyle = '--')

plt.scatter(x1, x2, c = y)
plt.show()

(e) Indicate the support vectors for the maximal margin classifier.

(f) Argue that a slight movement of the seventh observation would
not affect the maximal margin hyperplane.

ANS: 
(e) (2,1), (2,2), (4,3), (4,4).

(f) Obs(7) is (4,1) which is not support vectors so it would not affect the  maximal margin hyperplane.


(g) Sketch a hyperplane that is not the optimal separating hyper-
plane, and provide the equation for this hyperplane.

(h) Draw an additional observation on the plot so that the two
classes are no longer separable by a hyperplane.

In [ ]:
#(g)
plt.scatter(x1, x2, c = c)
x = np.arange(0, 5, 0.1)
y = 0.2*x + 0.5
plt.plot(x,y)
plt.show()

In [ ]:
#(h)
x1 = [3, 2, 4, 1, 2, 4, 4,3]
x2 = [4, 2, 4, 4, 1, 3, 1,1]
y = ["red", "red", "red", "red", "blue", "blue", "blue",'red']
plt.scatter(x1, x2, c = y)
plt.show()